##You will need a CUDA enabled GPU installed to run this code##

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import ast  # For safely evaluating string representations

# Load the CSV file
labels_df = pd.read_csv('encoded_labels.csv')

# Columns in your CSV
file_column = 'index'  # Replace with your actual filename column name
label_column = 'labels'   # Replace with your actual labels column name

# Directory where your images are stored
image_directory = './images/'  # Ensure this path ends with a '/'

# Function to convert string representation to list of integers
def parse_labels(label_str):
    # Replace spaces with commas to create a valid Python list string
    label_str = label_str.replace(' ', ',')
    # Use ast.literal_eval to safely evaluate the string to a list
    return list(map(int, ast.literal_eval(label_str)))

# Create full paths to images and parse labels
image_paths = [os.path.join(image_directory, fname) for fname in labels_df[file_column]]
labels = [parse_labels(label) for label in labels_df[label_column]]

# Split the dataset into training and validation sets
train_image_paths, val_image_paths, train_labels, val_labels = train_test_split(
    image_paths,
    labels,
    test_size=0.2,  # 20% for validation
    random_state=42
)


In [2]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torch

# Define transformations for preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match ResNet-50 input size
    transforms.ToTensor(),          # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with pre-trained model stats
])

# Define the dataset class
class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert('RGB')
        label = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        return img, torch.tensor(label, dtype=torch.float)  # Convert list to tensor

# Create datasets
train_dataset = CustomDataset(image_paths=train_image_paths, labels=train_labels, transform=transform)
val_dataset = CustomDataset(image_paths=val_image_paths, labels=val_labels, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [3]:
import torchvision.models as models
import torch.nn as nn

# Load a pre-trained ResNet-50 model
model = models.resnet50(pretrained=True)

# Modify the final fully connected layer for 14 classes
num_classes = 14
model.fc = nn.Linear(model.fc.in_features, num_classes)


c:\Users\Matt\anaconda3\envs\final_project\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Matt\anaconda3\envs\final_project\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
import torch.optim as optim

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # For multi-label classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Set the model to training mode
model.train()

num_epochs = 10  # Adjust the number of epochs as needed

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')


Epoch 1, Loss: 0.18481914825895998
Epoch 2, Loss: 0.17457938727968975
Epoch 3, Loss: 0.17181613544623056
Epoch 4, Loss: 0.1712415711359775
Epoch 5, Loss: 0.1686299256716214
Epoch 6, Loss: 0.16638082778411553
Epoch 7, Loss: 0.16483126342930693
Epoch 8, Loss: 0.16331722555642433
Epoch 9, Loss: 0.16042521199647417
Epoch 10, Loss: 0.15734446989306322


In [5]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images)
        predictions = torch.sigmoid(outputs)  # Apply sigmoid to get probabilities
        
        # Convert probabilities to binary predictions
        predicted_labels = (predictions > 0.5).float()
        
        # Calculate accuracy for each image
        correct += (predicted_labels == labels).all(dim=1).sum().item()
        total += labels.size(0)

print(f'Validation Accuracy: {100 * correct / total}%')

Validation Accuracy: 51.60427807486631%


In [6]:
torch.save(model.state_dict(), 'resnet50_finetuned.pth')
